<a href="https://colab.research.google.com/github/ethanmjansen/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/LS_DS10_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

#Pre-Work

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

#Train/Validate/Test Split

In [0]:
#Inspecting what I'm working with
df

In [0]:
#Converting Date to pandas datetime
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

In [0]:
#Replacing Boolean with 1 and 0 
df['Great'] = df['Great'].astype(int)
df.dtypes

In [0]:
#Making train 
train = df[df['Date'] <= '2017']
train

In [0]:
#Making validate 
val = df[(df['Date'] >= '2017') & (df['Date'] <= '2018')]
val

In [0]:
#Making test droping the y variable
test = df[df['Date'] >= '2018'] 
#test = test.drop('Great', axis=1)

In [242]:
#Inspecting shapes of original and comparing to my 3 way split
df.shape, train.shape, val.shape, test.shape

((421, 59), (298, 59), (85, 59), (38, 59))

#Baselines for Classification

In [0]:
#Import Baseline metric
from sklearn.metrics import accuracy_score

In [244]:
#Defining target and making y_train
target = 'Great'
y_train = train[target]
#Making y_val
y_val = val[target]
#Train Value Counts
print('Train')
print(y_train.value_counts(normalize=True)) # 0 = False and 1 = True
#Val value Counts 
print('Val')
print(y_val.value_counts(normalize=True)) # 0 = False and 1 = True

Train
0    0.590604
1    0.409396
Name: Great, dtype: float64
Val
0    0.552941
1    0.447059
Name: Great, dtype: float64


In [0]:
#Finding Majority Class for Train
majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

In [246]:
#Accuracy of Train
accuracy_score(y_train, y_pred)

0.5906040268456376

In [0]:
#Finding Majority Class for Val
majority_class = y_val.mode()[0]
y_pred = [majority_class] * len(y_val)

In [248]:
#Accurary of Val
accuracy_score(y_val, y_pred)

0.5529411764705883

#Using scikit-learn for logistical regression and validation accuracy

In [249]:
df.head(2)

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,2016-01-18,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,California,2016-01-24,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [0]:
#Import Estimator Class
from sklearn.linear_model import LogisticRegression

In [0]:
#Instantiate the Class
log_reg = LogisticRegression(solver='lbfgs')

In [0]:
#X Matrix and y vector
features = ['Cost', 'Hunger', 'Google']
X_train = train[features]
X_val = val[features]

# Impute missing values
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
log_reg.fit(X_train_imputed, y_train)

In [0]:
#Fit model 
log_reg.fit(X_train_imputed, y_train)

In [254]:
#Not optimal features
print('Validation Accuracy', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 0.5176470588235295


#Simulating a workflow 

In [0]:
#scikit-learn workflow 
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler

features = ['Burrito', 'Yelp', 'Google', 'Cost', 'Hunger', 'Meat', 'Fillings']
target = 'Great'
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [256]:
#Checking for continuity
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((298, 7), (298,), (85, 7), (85,))

In [0]:
#One-Hot Encoding everything
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)

In [0]:
#Imputing missing values
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)

In [0]:
#Scaling everything
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

In [273]:
#Fitting Model and getting score
model = LogisticRegressionCV(cv=5, n_jobs=-1, random_state=42)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))

Validation Accuracy 0.8235294117647058


#Model's Test Accuracy 


In [0]:
#Simulated workflow shortened
X_test = test[features]
X_test_encoded = encoder.transform(X_test)
X_test_imputed = imputer.transform(X_test_encoded)
X_test_scaled = scaler.transform(X_test_imputed)
y_pred = model.predict(X_test_scaled)

In [275]:
y_test = test[target]
print('Test Accuracy', model.score(X_test_scaled, y_test))

Test Accuracy 0.6578947368421053
